# 03_Connect_API_Banco_esp

Este notebook tem o objetivo de realizar uma conexão com o Spotify por meio das minhas credenciais de acesso, que podem ser obtidas através do link [fornecido](https://developer.spotify.com/dashboard/b8879123eaf64ef4b7d64c86afb417fd/settings).

Essa conexão permite a extração de informações sobre faixas e artistas, utilizando códigos específicos de faixas e artistas.

Com os códigos obtidos dos dados do meu Spotify, consigo acessar informações detalhadas sobre o meu estilo musical.


A inspiração para a utilização dessa ferramenta veio do artigo [Consumindo a API do Spotify: um breve passo a passo](https://medium.com/reprogramabr/consumindo-a-api-do-spotify-um-breve-passo-a-passo-fd210312fdd).


> Referências

- ReprogramaBR. (2020). Consumindo a API do Spotify: um breve passo a passo. Medium. [Link](https://medium.com/reprogramabr/consumindo-a-api-do-spotify-um-breve-passo-a-passo-fd210312fdd)


<a id='indice'></a>
## Indice

* [0. Importa bibliotecas e Variáveis de Ambiente](#0)
* [1. Configuração de conexão Pyspark](#1)
* [2. Configuração de conexão ao Spotify](#2)
* [3. Coleta de dados: Banco de dados](#3)
* [4. Extração e Upload](#4)

<a id='0'></a>
---

[Retorna ao Índice](#indice)

## 0. Importa bibliotecas e Variáveis de Ambiente

In [1]:
%load_ext autoreload
%autoreload 2

# Importar bibliotecas padrão
import time
from datetime import timedelta
import os

# Carregar variáveis de ambiente
from dotenv import load_dotenv
load_dotenv(os.path.join('config', '.env'))

# Importar bibliotecas para PostgreSQL
import psycopg2

# Importar bibliotecas para Spotify API
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import src.spotify_data as spotify_data

# Importar bibliotecas para PySpark
from pyspark.sql import SparkSession


user_name = 'User_E'
user = True if user_name == 'User_E'  else False
start_time_geral = time.time()

<a id='1'></a>
---

[Retorna ao Índice](#indice)

## 1. Configuração de conexão Pyspark

In [2]:
# Configurar SparkSession
spark = SparkSession.builder \
    .appName("PostgreSQL to Spark") \
    .config("spark.jars", "/caminho/para/postgresql-<versao>.jar") \
    .getOrCreate()


In [3]:
if user:
    db_config = {
        'host': os.getenv('DB_HOST'),
        'port': os.getenv('DB_PORT'),
        'dbname': os.getenv('DB_NAME'),
        'user': os.getenv('DB_USER'),
        'password': os.getenv('DB_PASSWORD')
    }

else:
    db_config = {
        'host': os.getenv('DB_HOST'),
        'port': os.getenv('DB_PORT'),
        'dbname': os.getenv('DB_NAME_2'),
        'user': os.getenv('DB_USER'),
        'password': os.getenv('DB_PASSWORD')
        }

# URL de conexão JDBC
jdbc_url = f"jdbc:postgresql://{db_config['host']}:{db_config['port']}/{db_config['dbname']}"

# Propriedades de conexão
connection_properties = {
    "user": db_config['user'],
    "password": db_config['password'],
    "driver": "org.postgresql.Driver"
}

<a id='2'></a>
---

[Retorna ao Índice](#indice)

## 2. Configuração de conexão ao Spotify

In [4]:
# Carregar variáveis de ambiente do arquivo .env na pasta config 
if user:
    client_id = os.getenv('CLIENT_ID') 
    client_secret = os.getenv('CLIENT_SECRET')
else:
    client_id = os.getenv('CLIENT_ID_2') 
    client_secret = os.getenv('CLIENT_SECRET_2')

# Configuração da autenticação
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

<a id='3'></a>
---

[Retorna ao Índice](#indice)
## 3. Coleta de dados: Banco de dados

In [5]:
start_time = time.time()

In [6]:
df = spark.read.jdbc(url=jdbc_url, table="all_tracks_registry", properties=connection_properties)
df.show(4)


+-------------------+--------------------+---------+------------+--------------+--------------------------+---------------------------------+--------------------------------+--------------------+------------+-----------------+-------------------+------------+----------+-------+-------+-------+-----------------+--------------+
|                 ts|            platform|ms_played|conn_country|       ip_addr|master_metadata_track_name|master_metadata_album_artist_name|master_metadata_album_album_name|   spotify_track_uri|episode_name|episode_show_name|spotify_episode_uri|reason_start|reason_end|shuffle|skipped|offline|offline_timestamp|incognito_mode|
+-------------------+--------------------+---------+------------+--------------+--------------------------+---------------------------------+--------------------------------+--------------------+------------+-----------------+-------------------+------------+----------+-------+-------+-------+-----------------+--------------+
|2019-11-11 00:1

In [7]:
df.count()

74704

In [8]:
df.printSchema()

root
 |-- ts: timestamp (nullable = true)
 |-- platform: string (nullable = true)
 |-- ms_played: integer (nullable = true)
 |-- conn_country: string (nullable = true)
 |-- ip_addr: string (nullable = true)
 |-- master_metadata_track_name: string (nullable = true)
 |-- master_metadata_album_artist_name: string (nullable = true)
 |-- master_metadata_album_album_name: string (nullable = true)
 |-- spotify_track_uri: string (nullable = true)
 |-- episode_name: string (nullable = true)
 |-- episode_show_name: string (nullable = true)
 |-- spotify_episode_uri: string (nullable = true)
 |-- reason_start: string (nullable = true)
 |-- reason_end: string (nullable = true)
 |-- shuffle: boolean (nullable = true)
 |-- skipped: boolean (nullable = true)
 |-- offline: boolean (nullable = true)
 |-- offline_timestamp: timestamp (nullable = true)
 |-- incognito_mode: boolean (nullable = true)



In [9]:
end_time = time.time()
elapsed_time = end_time - start_time
# Converter o tempo decorrido para o formato HH:MM:SS 
print(f"Tempo de processamento: {str(timedelta(seconds=int(elapsed_time)))}")

Tempo de processamento: 0:00:06


<a id='4'></a>
---

[Retorna ao Índice](#indice)
## 4. Extração e Upload

In [10]:
start_time = time.time()

In [11]:
# Funções de banco de dados Tracks e Artistics
import src.db_table_track as db_table_track
import src.db_table_artistcs as db_table_artistcs

In [12]:
# Selecionar e coletar os dados distintos 
distinct_uris = df.select('spotify_track_uri').distinct().collect() 
# Transformar em lista de valores simples 
uris_list = [row['spotify_track_uri'] for row in distinct_uris]
print(f'Quantidade de faixas: {len(uris_list)}')

Quantidade de faixas: 14403


In [13]:
# n = 1560
n = 230
for track_uri in uris_list[n:]:
    print(f'Faixa número:{n}')
    # Primeiro Verifica se o track já existe na tabela, se FALSE, realiza requisição no spotify
    if db_table_track.track_exists(track_uri, conn=psycopg2.connect(**db_config)):
        print(f'n:{n}, TRACK Já existe no banco')
        main_artist_uri = db_table_track.get_main_artist_uri(track_uri, conn=psycopg2.connect(**db_config))

    else:
        # Realiza requisição da faixa e depois inseri no banco
        dados_faixa = spotify_data.obter_dados_faixa(sp = sp, track_uri = track_uri)
        db_table_track.insert_track_data(dados_faixa, conn=psycopg2.connect(**db_config))

        # Coleta de dados do Artista no Spotify
        main_artist_uri = dados_faixa['main_artist_URI']
        time.sleep(5)

    if db_table_artistcs.artist_exists(main_artist_uri, conn=psycopg2.connect(**db_config)):
        # Não fazer nada se o artista já existir
        print(f'n:{n}, ARTISTIC Já existe no banco')
        pass
    else:
        # Realiza requisição do artista e depois inseri no banco
        dados_artista = spotify_data.obter_dados_artista(sp=sp, artist_uri = main_artist_uri)
        db_table_artistcs.insert_artist_data(dados_artista, conn=psycopg2.connect(**db_config))
    
    n = n + 1  

Faixa número:0
Faixa número:1
Faixa número:2
Faixa número:3
Faixa número:4
Faixa número:5
Faixa número:6
Faixa número:7
Faixa número:8
Faixa número:9
Faixa número:10
Faixa número:11
Faixa número:12
Faixa número:13
Faixa número:14
Faixa número:15
Faixa número:16
Faixa número:17
Faixa número:18
Faixa número:19
n:19, ARTISTIC Já existe no banco
Faixa número:20
Faixa número:21
Faixa número:22
Faixa número:23
Faixa número:24
Faixa número:25
Faixa número:26
Faixa número:27
Faixa número:28
Faixa número:29
Faixa número:30
Faixa número:31
Faixa número:32
Faixa número:33
Faixa número:34
Faixa número:35
Faixa número:36
Faixa número:37
Faixa número:38
Faixa número:39
Faixa número:40
Faixa número:41
Faixa número:42
Faixa número:43
Faixa número:44
Faixa número:45
Faixa número:46
Faixa número:47
Faixa número:48
Faixa número:49
Faixa número:50
Faixa número:51
Faixa número:52
Faixa número:53
Faixa número:54
Faixa número:55
Faixa número:56
Faixa número:57
Faixa número:58
Faixa número:59
Faixa número:60


KeyboardInterrupt: 

In [ ]:
end_time = time.time()
elapsed_time = end_time - start_time
# Converter o tempo decorrido para o formato HH:MM:SS 
print(f"Tempo de processamento: {str(timedelta(seconds=int(elapsed_time)))}")


In [ ]:
# import pyspark.sql.functions as F
# df = df.withColumn('ano-mes-dia', F.to_date(F.col("ts")))
# df = df.withColumn("ano-mes", F.concat_ws("-", F.year(df["ts"]), F.month(df["ts"])))
# df = df.withColumn("ano_mes_str", F.date_format(df["ts"], "yyyy-MM")) 
# df = df.withColumn("ano_mes_date", F.to_date(df["ano_mes_str"], "yyyy-MM"))
# df.show(5, truncate=False)